# Hastings Direct Takehome

Background:
Insurance companies make pricing decisions based on historical claims experience. The more recent the claims experience, the more predictive it may be of future losses. In the case of many large claims however, the exact cost is not known at the time of the accident. In fact, some cases take years to develop and settle. Companies sometimes learn that a claim is large several years after the accident took place.
Your Underwriting Director believes it is possible to predict the ultimate value of individual claims well in advance by using FNOL (First Notification Of Loss) characteristics. This is the information recorded when the claim is first notified. If so, it would allow the company to know about future costs earlier and this information could be used to make better pricing decisions.
You are given a historical dataset of a particular type of claim - head-on collisions - and are also told their individual current estimated values (labelled Incurred). (Given these claims are now a few years old, you can assume the incurred values are equal to the cost at which the claims will finally settle). 

Task breakdown:
1) Using this data, build a model to predict the ultimate individual claim amounts
"2) Prepare a 15 minute presentation summarising your model. Your presentation should either be in notebook format or a more traditional slide deck.  If you opt for the slide deck approach, please make sure that you provide supporting code. 
Your presentation should cover the following aspects:
- Issues identified with the data and how these were addressed
- Data cleansing
- Model specification and justification for selecting this model specification
- Assessment of your model's accuracy and model diagnostics
- Suggestions of how your model could be improved
- Practical challenges for implementing your model"

Note: columns beginning with TP_* show the number of third parties involved in an accident (under a given category)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport


/var/folders/g7/_7cggn5x0b3bf_rty1bqjb3c0000gn/T/ipykernel_32974/2179082641.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
data = pd.read_csv('data/task_data.csv')

In [3]:
data.head(3).T

,0,1,2
Claim Number,1,2,3
date_of_loss,2003-04-15,2003-04-20,2003-04-24
Notifier,PH,CNF,CNF
Loss_code,LD003,LD003,LD003
Loss_description,Head on collision,Head on collision,Head on collision
Notification_period,22,1,5
Inception_to_loss,13,9,17
Location_of_incident,Main Road,Main Road,Main Road
Weather_conditions,NORMAL,WET,WET
Vehicle_mobile,Y,Y,Y


In [4]:
print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)}")

Dataset shape: (7691, 47)
Columns: ['Claim Number', 'date_of_loss', 'Notifier', 'Loss_code', 'Loss_description', 'Notification_period', 'Inception_to_loss', 'Location_of_incident', 'Weather_conditions', 'Vehicle_mobile', 'Time_hour', 'Main_driver', 'PH_considered_TP_at_fault', 'Vechile_registration_present', 'Incident_details_present', 'Injury_details_present', 'TP_type_insd_pass_back', 'TP_type_insd_pass_front', 'TP_type_driver', 'TP_type_pass_back', 'TP_type_pass_front', 'TP_type_bike', 'TP_type_cyclist', 'TP_type_pass_multi', 'TP_type_pedestrian', 'TP_type_other', 'TP_type_nk', 'TP_injury_whiplash', 'TP_injury_traumatic', 'TP_injury_fatality', 'TP_injury_unclear', 'TP_injury_nk', 'TP_region_eastang', 'TP_region_eastmid', 'TP_region_london', 'TP_region_north', 'TP_region_northw', 'TP_region_outerldn', 'TP_region_scotland', 'TP_region_southe', 'TP_region_southw', 'TP_region_wales', 'TP_region_westmid', 'TP_region_yorkshire', 'Incurred', 'Capped Incurred', 'Unnamed: 46']


## Data Cleaning

Lots of tab characters messing things up

In [16]:
cols_to_strip = data.columns[data.columns.get_loc('Vechile_registration_present'):]
pattern = r'[^0-9]'
data[cols_to_strip] = data[cols_to_strip].apply(lambda col: col.astype(str).str.replace(pattern, '', regex=True))

Fill in missing 0s in flag cols

In [22]:
flag_cols = data.columns[data.columns.get_loc('Vechile_registration_present'): data.columns.get_loc('Incurred')]
data[flag_cols] = (
    data[flag_cols]
    .replace('', np.nan)
    .fillna(0)
    .astype('int64')
)

/var/folders/g7/_7cggn5x0b3bf_rty1bqjb3c0000gn/T/ipykernel_32974/1260033915.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('', np.nan)


Convert currency cols

In [24]:
currency_cols = data.columns[data.columns.get_loc('Incurred'):]
data[currency_cols] = data[currency_cols].replace('', np.nan).astype('float64')

/var/folders/g7/_7cggn5x0b3bf_rty1bqjb3c0000gn/T/ipykernel_32974/3113443979.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[currency_cols] = data[currency_cols].replace('', np.nan).astype('float64')


Can drop empty or constant (all the same) columns

In [27]:
cols_to_drop = [
    'Unnamed: 46'
    ,'Loss_code' # all the same
    ,'Loss_description' # all the same
]
data = data.drop(cols_to_drop, axis=1)

In [29]:
data['date_of_loss'] = pd.to_datetime(data['date_of_loss'])

Weather condition has some missing values, think we can use n/k for these safely

In [35]:
data['Weather_conditions'].value_counts(dropna=False)

Weather_conditions
NORMAL          4564
WET             1903
N/K              450
SNOW,ICE,FOG     429
NaN              345
Name: count, dtype: int64

In [37]:
data['Weather_conditions'] = data['Weather_conditions'].fillna('N/K')
data['Weather_conditions'].value_counts(dropna=False)

Weather_conditions
NORMAL          4564
WET             1903
N/K              795
SNOW,ICE,FOG     429
Name: count, dtype: int64

In [30]:
data.head(3).T

,0,1,2
Claim Number,1,2,3
date_of_loss,2003-04-15 00:00:00,2003-04-20 00:00:00,2003-04-24 00:00:00
Notifier,PH,CNF,CNF
Notification_period,22,1,5
Inception_to_loss,13,9,17
Location_of_incident,Main Road,Main Road,Main Road
Weather_conditions,NORMAL,WET,WET
Vehicle_mobile,Y,Y,Y
Time_hour,10,18,16
Main_driver,Other,Other,Y


In [31]:
data.dtypes

Claim Number                             int64
date_of_loss                    datetime64[ns]
Notifier                                object
Notification_period                      int64
Inception_to_loss                        int64
Location_of_incident                    object
Weather_conditions                      object
Vehicle_mobile                          object
Time_hour                                int64
Main_driver                             object
PH_considered_TP_at_fault               object
Vechile_registration_present             int64
Incident_details_present                 int64
Injury_details_present                   int64
TP_type_insd_pass_back                   int64
TP_type_insd_pass_front                  int64
TP_type_driver                           int64
TP_type_pass_back                        int64
TP_type_pass_front                       int64
TP_type_bike                             int64
TP_type_cyclist                          int64
TP_type_pass_

In [ ]:
profile = ProfileReport(
    data, 
    title="Hastings Direct Claims Data - Pandas Profiling Report",
    explorative=True,
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": True},
        "phi_k": {"calculate": True},
        "cramers": {"calculate": True}
    },
    missing_diagrams={
        "matrix": True,
        "bar": True,
        "heatmap": True,
        "dendrogram": True
    },
    duplicates={
        "head": 10
    }
)

Generating pandas profiling report...


In [11]:
output_file = "hastings_data_profile.html"
profile.to_file(output_file)

Saving report to hastings_data_profile.html...


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Report saved as: hastings_data_profile.html


In [ ]:
category_cols = [
    'Notifier'
    ,'Location_of_incident'
    ,'Weather_conditions'
    ,'Vehicle_mobile'
]

Qs
What does inception to loss mean?

In [ ]:
drop_cols = [
    'Loss_code' # all the same
    ,'Loss_description' # all the same
]

In [19]:
data.dtypes

Claim Number                      int64
date_of_loss                     object
Notifier                         object
Loss_code                        object
Loss_description                 object
Notification_period               int64
Inception_to_loss                 int64
Location_of_incident             object
Weather_conditions               object
Vehicle_mobile                   object
Time_hour                         int64
Main_driver                      object
PH_considered_TP_at_fault        object
Vechile_registration_present     object
Incident_details_present         object
Injury_details_present           object
TP_type_insd_pass_back           object
TP_type_insd_pass_front          object
TP_type_driver                   object
TP_type_pass_back                object
TP_type_pass_front               object
TP_type_bike                     object
TP_type_cyclist                  object
TP_type_pass_multi               object
TP_type_pedestrian               object


TO DO:
Weather_conditions needs missing filling - us N/K value
Time_hour has suspicious amount at 0 - probably missing value so treat that as categorical
Vechile_registration_present has only 2 distinct values and nearly all are 1 except 6 whihc are -.

In [23]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr, spearmanr

def explore_col(df, col, target):
    summary = {
        "col": col,
        "target": target,
        "col_data_type": df[col].dtype.name
    }

    col_data = df[col]
    target_data = df[target]

    if col_data.dtype == 'object' or pd.api.types.is_categorical_dtype(col_data):
        encoded_col = LabelEncoder().fit_transform(col_data.astype(str))
        corr, _ = spearmanr(encoded_col, target_data)
        summary["col_target_corr"] = corr
        summary["n_unique"] = col_data.nunique()
        summary["top_categories"] = col_data.value_counts().head(5).to_dict()

        # Visualize mean target per category
        agg_df = df.groupby(col)[target].mean().reset_index().sort_values(by=target, ascending=False)
        fig = px.bar(agg_df, x=col, y=target, title=f"Mean {target} by {col}", template="plotly_white")

    else:
        corr, _ = pearsonr(col_data.fillna(0), target_data.fillna(0))
        summary["col_target_corr"] = corr
        summary["mean"] = col_data.mean()
        summary["std"] = col_data.std()

        if target_data.nunique() < 10:
            fig = px.box(df, x=target, y=col, points="all", title=f"{col} distribution by {target}", template="plotly_white")
        else:
            fig = px.scatter(df, x=col, y=target, trendline="ols", title=f"{col} vs {target} (Corr: {corr:.2f})", template="plotly_white")

    fig.update_layout(title_font_size=18, height=400, margin=dict(t=40, b=20, l=10, r=10))
    fig.show()

    return summary

In [24]:
for col in data.columns:
    explore_col(data, col, 'Incurred')

/var/folders/g7/_7cggn5x0b3bf_rty1bqjb3c0000gn/T/ipykernel_11659/1395919154.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col_data.dtype == 'object' or pd.api.types.is_categorical_dtype(col_data):


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U7')) -> None